# Datasets 2014
Para cada año tenemos 3 tablas a inspeccionar, limpiar y unir:
- `title_basics`: pública y descargada de IMDb
- `title_ratings`: pública y descargada de IMDb
- `movies`: propia, escrapeada de la web de IMDb para todas las películas del año correspondiente

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
anno = 2014

In [3]:
# Tabla descargada de IMDb

title_basics = pd.read_csv('../data/imdb/title_basics.tsv', sep='\t', na_values='\\N')
print(title_basics.shape)
title_basics.head()

C:\Users\casiopa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8084314, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [4]:
# Tabla descargada de IMDb

title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286


In [62]:
# Películas de escrapeadas de la web IMDb

movies = pd.read_csv('../data/web_imdb/movies_df_'+ str(anno) + '.csv', sep=';', na_values=["[]", "['']", "['', '']", "['', '', '']"])
print(movies.shape)
movies.head()

(15863, 20)


,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
0,tt0330987,"Tiden är en dröm, del 2",NaN,2014.0,NaN,NaN,['Jan Lindqvist'],"['Maths Isacson', 'Jan Lindqvist']",['Jan Lindqvist'],"['Documentary', 'History']",['Sweden'],"['Studio Ladan AB', 'Svenska Filminstitutet (S...",7.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0016906,Frivolinas,NaN,2014.0,Not Rated,1h 20min,['Arturo Carballo'],"['José López Alonso', 'Juan Belmonte', 'María ...",NaN,"['Comedy', 'Musical']",['Spain'],['Ediciones Seleccionadas Arturo Carballo'],5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0338258,Instruments of Darkness,NaN,2014.0,NaN,NaN,['Vincent Regan'],"['Vincent Regan', 'William Shakespeare']","['Sean Bean', 'Charles Dance', ""James D'Arcy""]","['Action', 'Drama', 'Fantasy']","['United Kingdom', 'France']",['Hell Kite Films'],6.6,NaN,NaN,NaN,"£5,400,000",NaN,NaN,NaN
3,tt0365907,Caminando entre las tumbas,A Walk Among the Tombstones,2014.0,18,1h 54min,['Scott Frank'],"['Lawrence Block', 'Scott Frank']","['Liam Neeson', 'Dan Stevens', 'David Harbour']","['Action', 'Crime', 'Drama']",['United States'],"['1984 Private Defense Contractors', 'Cross Cr...",6.5,57.0,"2,614",Awards\n1 nomination,"$28,000,000","$26,307,600","$12,758,780","$58,834,384"
4,tt0329539,Rice Girl,NaN,2014.0,NaN,NaN,['Michael Fischa'],['Cat Ling'],"['Pat Morita', 'Martin Kove', 'Ian Lithgow']","['Adventure', 'Comedy', 'Family']",['United States'],"['Alpha-O Production', 'Rice Girl Productions ...",5.8,NaN,NaN,Awards\n1 nomination,NaN,NaN,NaN,NaN


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15863 entries, 0 to 15862
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   imdb_id            15863 non-null  object 
 1   title              15862 non-null  object 
 2   original_title     623 non-null    object 
 3   year               15858 non-null  float64
 4   certificate        3429 non-null   object 
 5   duration           3409 non-null   object 
 6   directors          15530 non-null  object 
 7   writers            14504 non-null  object 
 8   stars              10902 non-null  object 
 9   genres             15371 non-null  object 
 10  countries          15524 non-null  object 
 11  companies          11017 non-null  object 
 12  imdb_rating        8908 non-null   float64
 13  metascore          871 non-null    float64
 14  popularity         160 non-null    object 
 15  awards             4078 non-null   object 
 16  budget             437

---
## Número de películas

En primer lugar verificar si hay una discrepancia entre el número de películas obtenidas de title_basics.tsv de IMDb
y el número de películas resultantes del scrapping.

In [65]:
# Cargar las películas de la base de datos de IMDb en una lista

imdb_ids = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]
imdb_ids = list(imdb_ids['tconst'])
len(imdb_ids)

15856

In [64]:
scrap_ids = list(movies['imdb_id'])
len(scrap_ids)

15863

In [75]:
# Eiminar películas con ids diferentes al listado de referencia

ids_to_erase= []
for id in scrap_ids:
    if id not in imdb_ids:
        ids_to_erase.append(id)
        
movies = movies[~movies.imdb_id.isin(ids_to_erase)]
len(movies)

15856

---
## 1. Bd de IMDb: `title_basics`

### Variable `runtimeMinutes`  

In [76]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15856 entries, 16657 to 8081106
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          15856 non-null  object 
 1   titleType       15856 non-null  object 
 2   primaryTitle    15856 non-null  object 
 3   originalTitle   15856 non-null  object 
 4   isAdult         15856 non-null  float64
 5   startYear       15856 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  13089 non-null  float64
 8   genres          15364 non-null  object 
dtypes: float64(4), object(5)
memory usage: 1.2+ MB


#### Convertir `runtimeMinutes` en float
La siguiente función convierte los valores en float si se puede de manera natural, y si no se puede los imprime y convierte a nulo.

In [77]:
def to_float(n):
    try:
        n = float(n)
        return n
    except:
        print(n)
        return np.nan

In [78]:
title_basics.runtimeMinutes = title_basics.runtimeMinutes.apply(to_float)
title_basics.runtimeMinutes

16657       80.0
315931      95.0
317334     109.0
324428       NaN
351045     114.0
           ...  
8066212      NaN
8071565    120.0
8071583      NaN
8077055     52.0
8081106     54.0
Name: runtimeMinutes, Length: 15856, dtype: float64

#### Eliminar de `title_basics` los registros con outliers en `runtimeMinutes`

In [79]:
# Buscar outliers mayores de 1.000 minutos y menores 15
# El resltado es un listado de películas fake, cortos o compendios de pelis.
# Eliminar todos estos registros del scrapping

title_basics = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]

title_basics = title_basics[~((title_basics.runtimeMinutes<15) | (title_basics.runtimeMinutes>500))]
title_basics.shape

(15856, 9)

In [80]:
# Una vez eliminados los outliers estos son los estadísticos:

title_basics.runtimeMinutes.describe().round(2)

count    13089.00
mean        87.91
std         26.51
min         41.00
25%         72.00
50%         88.00
75%        100.00
max        450.00
Name: runtimeMinutes, dtype: float64

---
## 2. Bd de IMDb: `title_ratings`

In [81]:
title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286



---
## 3. Bd propia escrapeada del portal IMDb: `movies`

## `year`
La base de datos de IMDb descargada tiene algunas discrepancias en cuanto al año de las películas, no coindice el año en la base de datos descargada de IMDb con el año del portal web, en el caso del año 2019 ocurre en 12 registros. Existe algún caso en el que tampoco coincide el año de la película en el portal Metacritic. Hay varias formas de afrontar estas discrepancias, en este caso, y por simplificación, utilizaré el año de la base de datos de IMDb.
Hay también algunos registros en la base de datos con valores nulos para el año y demás columnas, y serán eliminados (47 registors en total).

In [82]:
# Películas escrapeadas con año diferente
len(movies[(movies['year']!=anno) & movies['year'].notnull()])

4

In [83]:
movies[(movies['year']!=anno) & movies['year'].notnull()]

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
645,tt1286146,Frenchy,NaN,2021.0,NaN,NaN,['Jean-Claude Van Damme'],['Jean-Claude Van Damme'],"['Jean-Claude Van Damme', 'Claudia Bassols', '...","['Action', 'Crime', 'Drama']","['Thailand', 'Hong Kong', 'United States']","['Bomar OOD', 'Full Love Productions', 'Michae...",7.5,NaN,NaN,NaN,"$5,000,000",NaN,NaN,NaN
1795,tt2193079,Ghost Light,NaN,2013.0,Not Rated,1h 37min,['John Gaspard'],"['John Gaspard', 'Mary Kaeding']","['Amy Shomshak', 'Austan Peterschick', 'Dawn B...",['Comedy'],['United States'],NaN,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3515,tt2925810,"The Gun, the Cake and the Butterfly",NaN,2013.0,Not Rated,1h 15min,['Amanda Eliasch'],"['Amanda Eliasch', 'Lyall B. Watson']","['Justine Francesca Glenton', 'Elizabeth Karr'...","['Documentary', 'Biography', 'Drama']","['United Kingdom', 'United States']",['Butterfly Movies'],6.2,NaN,NaN,Awards\n2 wins,"£150,000",NaN,NaN,NaN
8211,tt3711938,"Chaparri, les sept ours de la montagne sacrée",NaN,2010.0,NaN,NaN,['Nathalie Granger-Charles-Dominique'],NaN,NaN,['Documentary'],['France'],['Hévadis Films'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Eliminar del df escrapeado las películas de años posteriores a 2019
(son pelis futuras que se han colado)

In [84]:
movies = movies[~(movies['year']>2019)]
len(movies)

15855

#### Eliminar del df escrapeado las películas sin año
(son pelis sin información o url en el portal)

In [85]:
# Películas escrapeadas con año null, no tienen casi información, serán eliminadas
movies = movies[movies['year'].notnull()]
len(movies)

15851

## `imdb_rating`
Esta variable será sustituida por la de la base de datos `title_rating` descargada de IMDb

In [86]:
movies[['imdb_rating']].describe().round(2)

,imdb_rating
count,8906.00
mean,6.26
std,1.45
min,1.00
25%,5.40
50%,6.40
75%,7.30
max,10.00


## `metascore`
Comparing to: https://www.metacritic.com/feature/best-movies-released-in-2019

Hay pequeñas discrepancias en el listado de películas, debido a las fechas de estreno. Metacritic considera la fecha de estreno en USA. IMDb la fecha de estreno mundial. Pero sí que coinciden las puntuaciones y la mayoría de películas asignadas a 2019.

In [87]:
movies[['imdb_id', 'title', 'original_title', 'imdb_rating', 'metascore', 'year']][movies.metascore>=82].sort_values('metascore', ascending=False)

,imdb_id,title,original_title,imdb_rating,metascore,year
141,tt1065073,Boyhood (Momentos de una vida),Boyhood,7.9,100.0,2014.0
5896,tt3455224,Virunga,NaN,8.2,95.0,2014.0
2510,tt2473794,Mr. Turner,NaN,6.8,94.0,2014.0
6482,tt3521134,La mirada del silencio,The Look of Silence,8.3,92.0,2014.0
5598,tt3409392,Timbuktu,NaN,7.2,92.0,2014.0
3223,tt2802154,Leviatán,Leviafan,7.6,92.0,2014.0
3972,tt3062880,Gett: El divorcio de Viviane Amsalem,Gett,7.7,90.0,2014.0
4908,tt3263996,The Overnighters,NaN,7.4,89.0,2014.0
3129,tt2764784,Phoenix,NaN,7.3,89.0,2014.0
2606,tt2516274,The Winding Stream,NaN,7.8,89.0,2014.0


In [88]:
np.sort(movies.metascore.dropna().unique())

array([  1.,   5.,   7.,   8.,   9.,  11.,  12.,  14.,  15.,  16.,  17.,
        18.,  19.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,
        30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
        41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,
        52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,
        63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,
        74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,
        85.,  86.,  87.,  88.,  89.,  90.,  92.,  94.,  95., 100.])

In [89]:
movies.metascore.describe()

count    871.000000
mean      55.516648
std       18.210314
min        1.000000
25%       42.000000
50%       56.000000
75%       70.000000
max      100.000000
Name: metascore, dtype: float64

## `popularity`
Valor que vamos a descartar porque es muy volátil, cambia continuamente. Así, no guarda relación temporal con la taquilla que es el objeto de estudio.

In [90]:
# Convertirlo a número
movies['popularity'] = movies.popularity.str.replace(',', '').astype(float)

In [91]:
movies.popularity.describe()

count     160.000000
mean     2715.237500
std      1450.945792
min       108.000000
25%      1498.250000
50%      2791.500000
75%      3989.750000
max      4973.000000
Name: popularity, dtype: float64

## `directors`, `writers`, `stars`, `countries`, `companies`
Convertir a: comma separated values

In [92]:
movies['directors'] = movies['directors'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [93]:
movies['writers'] = movies['writers'].str.replace('[', '', regex=False) \
                                     .str.replace(']', '', regex=False) \
                                     .str.replace("'", "", regex=False) \
                                     .str.replace(", ", ",", regex=False)

In [94]:
movies['stars'] = movies['writers'].str.replace('[', '', regex=False) \
                                   .str.replace(']', '', regex=False) \
                                   .str.replace("'", "", regex=False) \
                                   .str.replace(", ", ",", regex=False)

In [95]:
movies['countries'] = movies['countries'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [96]:
movies['companies'] = movies['companies'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

## `opening_us_canada`, `gross_us_canada`, `gross_world`
Convertir estos valores a números, eliminando el signo del dólar

In [97]:
def remove_dolar(amount):
    if str(amount) == 'nan':
        return np.nan
    else:        
        return float(amount.replace(',', '')[1:]) 

In [98]:
movies['gross_us_canada'] = movies['gross_us_canada'].apply(remove_dolar)
movies['gross_us_canada']

0               NaN
1               NaN
2               NaN
3        26307600.0
4               NaN
            ...    
15858           NaN
15859           NaN
15860           NaN
15861           NaN
15862           NaN
Name: gross_us_canada, Length: 15851, dtype: float64

In [99]:
movies['opening_us_canada'] = movies['opening_us_canada'].apply(remove_dolar)
movies['opening_us_canada']

0               NaN
1               NaN
2               NaN
3        12758780.0
4               NaN
            ...    
15858           NaN
15859           NaN
15860           NaN
15861           NaN
15862           NaN
Name: opening_us_canada, Length: 15851, dtype: float64

In [100]:
movies['gross_world'] = movies['gross_world'].apply(remove_dolar)
movies['gross_world']

0               NaN
1               NaN
2               NaN
3        58834384.0
4               NaN
            ...    
15858           NaN
15859           NaN
15860           NaN
15861           NaN
15862           NaN
Name: gross_world, Length: 15851, dtype: float64

## `budget`
### Normalización `budget` a dólares
Son necesarios varios procesos para tener un valor válido de la variable `budget`:
- Separar, con regex, el texto que corresponde a la moneda y el que corresponde a la cantidad (ej. '€195,000'). Estos dos textos serán dos nuevas columnas `budgetCurrency` y `budgetAmount`
- Crear un diccionario con la correspondencia del código usado para las monedas en IMDb y el código ISO
- Crear un archivo externo que contenga las tasas de cambio para cada moneda y año
- Crear una nueva columna `budgetInDolars` con el valor ya normalizado a dólares

In [101]:
import re

def clean_amount_currency(raw_amount):
    
    if str(raw_amount) == 'nan' :
        return (np.nan, np.nan)
    
    # Diccionario con correspondencia del símbolo de moneda de IMDb y el ISO de monedas
    # Monedas de 2019 sin tasas de cambio: 'LVL'
    currency_to_iso = {'$':'USA', 'A$':'AUS', 'ZAR':'ZAF', '₹':'IND', '€':'EU27_2020',
                       'RUR':'RUS', 'R$':'BRA', 'CN¥':'CHN', 'TRL':'TUR', 'CA$':'CAN',
                       'NPR':'NPR', '£':'GBR', '¥':'JPN', 'HUF':'HUN', 'HRK':'HRV',
                       'NOK':'NOR', 'MNT':'MNT', 'CHF':'CHE', 'MX$':'MEX', 'UAH':'UAH',
                       'HK$':'HKG', 'PHP':'PHP', 'PLN':'POL', 'EGP':'EGP' , 'IRR':'IRR',
                       'NZ$':'NZL', 'AED':'AED', 'DKK':'DNK', 'NGN':'NGN', 'SEK':'SWE',
                       'BDT':'BDT', 'MYR':'MYR', 'CZK':'CZE', 'ARS':'ARG', 'PKR':'PKR',
                       'GEL':'GEL', 'NT$':'TWD', '₪':'ISR', '₫':'VND', '₩':'KOR', 'THB':'THB',
                       'RON':'ROU', 'PTE':'PRT', 'TTD':'TTD', 'MVR':'MVR', 'JOD':'JOD',
                       'ISK':'ISL', 'COP':'COL', 'LKR':'LKR', 'BSD':'BSD', 'BGL':'BGR',
                       'KZT':'KZT', 'KES':'KES', 'TZS':'TZS', 'DOP':'DOP', 'TJS':'TJS',
                       'TND':'TND', 'MUR':'MUR', 'SGD':'SGD', 'MOP':'MOP', 'IDR':'IDN',
                       'ETB':'ETB', 'CUP':'CUP', 'BYR':'BYR', 'MMK':'MMK', 'NAD':'NAD',
                       'PEN':'PER', 'MKD':'MKD', 'UGX':'UGX', 'BAM':'BAM', 'ALL':'ALL',
                       'KGS':'KGS', 'GHC':'GHS'
                      }
    
    raw_amount = raw_amount.replace(',', '')
    
    for item in re.finditer(r"^(\D+)(\d+)$", raw_amount):
        try:
            currency = currency_to_iso[item.group(1).strip()]
        except:
            currency = str(np.nan)
        amount = float(item.group(2))

    return(currency, amount)



# Dividir la info de budget en dos nuevas columnas (cantidad y moneda)
movies['budgetCurrency'], movies['budgetAmount'] = zip(*movies['budget'].map(clean_amount_currency))
movies[['title', 'budget', 'budgetAmount', 'budgetCurrency']].dropna()

# importar archivo csv de tasas de cambio
exchange_rates=pd.read_csv('../data/OECD/DP_LIVE_16072021155836489.csv')

# Seleccionar las tasas de cambio para el año del dataset
exchange_rates_anno = exchange_rates[exchange_rates.TIME == anno]

# Merge tasas de cambio con df movies, usando como primary key el código ISO de la moneda
movies = pd.merge(movies, exchange_rates_anno[['LOCATION','Value']], how='left', left_on='budgetCurrency', right_on='LOCATION')
movies['budgetInDollars'] = movies['budgetAmount'] / movies['Value']

movies.head()

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,awards,budget,gross_us_canada,opening_us_canada,gross_world,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars
0,tt0330987,"Tiden är en dröm, del 2",NaN,2014.0,NaN,NaN,Jan Lindqvist,"Maths Isacson,Jan Lindqvist","Maths Isacson,Jan Lindqvist","['Documentary', 'History']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0016906,Frivolinas,NaN,2014.0,Not Rated,1h 20min,Arturo Carballo,"José López Alonso,Juan Belmonte,María Caballé","José López Alonso,Juan Belmonte,María Caballé","['Comedy', 'Musical']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0338258,Instruments of Darkness,NaN,2014.0,NaN,NaN,Vincent Regan,"Vincent Regan,William Shakespeare","Vincent Regan,William Shakespeare","['Action', 'Drama', 'Fantasy']",...,NaN,"£5,400,000",NaN,NaN,NaN,GBR,5400000.0,GBR,0.60773,8.885525e+06
3,tt0365907,Caminando entre las tumbas,A Walk Among the Tombstones,2014.0,18,1h 54min,Scott Frank,"Lawrence Block,Scott Frank","Lawrence Block,Scott Frank","['Action', 'Crime', 'Drama']",...,Awards\n1 nomination,"$28,000,000",26307600.0,12758780.0,58834384.0,USA,28000000.0,USA,1.00000,2.800000e+07
4,tt0329539,Rice Girl,NaN,2014.0,NaN,NaN,Michael Fischa,Cat Ling,Cat Ling,"['Adventure', 'Comedy', 'Family']",...,Awards\n1 nomination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
# Merge de tablas


In [102]:
imdb_merged = pd.merge(title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)] , title_ratings, on='tconst', how='left')
imdb_merged

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0016906,movie,Frivolinas,Frivolinas,0.0,2014.0,NaN,80.0,"Comedy,Musical",5.6,15.0
1,tt0329539,movie,Rice Girl,Rice Girl,0.0,2014.0,NaN,95.0,"Adventure,Comedy,Family",5.8,86.0
2,tt0330987,movie,"Tiden är en dröm, del 2","Tiden är en dröm, del 2",0.0,2014.0,NaN,109.0,"Documentary,History",7.2,13.0
3,tt0338258,movie,Instruments of Darkness,Instruments of Darkness,0.0,2014.0,NaN,NaN,"Action,Drama,Fantasy",6.6,25.0
4,tt0365907,movie,A Walk Among the Tombstones,A Walk Among the Tombstones,0.0,2014.0,NaN,114.0,"Action,Crime,Drama",6.5,114950.0
...,...,...,...,...,...,...,...,...,...,...,...
15851,tt9878814,movie,La neve nera di Oslo,La neve nera di Oslo,0.0,2014.0,NaN,NaN,Documentary,NaN,NaN
15852,tt9890070,movie,Maaya,Maaya,0.0,2014.0,NaN,120.0,Thriller,6.9,34.0
15853,tt9890106,movie,Boy Meets Girl (Tholiprema Katha),Boy Meets Girl (Tholiprema Katha),0.0,2014.0,NaN,NaN,Romance,NaN,NaN
15854,tt9901656,movie,Entire Life and One Day,Entire Life and One Day,0.0,2014.0,NaN,52.0,Documentary,NaN,NaN


### Renombrar y seleccionar columnas de ambos datasets

In [103]:
imdb_merged.rename(columns={'primaryTitle': 'englishTitle',
                            'tconst': 'imdbId',
                            'averageRating': 'ratingImdb'
                          }, inplace=True)

In [104]:
imdb_merged.columns

Index(['imdbId', 'titleType', 'englishTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres', 'ratingImdb',
       'numVotes'],
      dtype='object')

In [105]:
movies.rename(columns={'imdb_id': 'imdbId',
                       'title': 'spanishTitle',
                       'budget': 'budgetString',
                       'budgetInDollars': 'budget',
                       'gross_us_canada': 'grossUsCanada',
                       'opening_us_canada': 'openingUsCanada',
                       'gross_world': 'grossWorld',
                      }, inplace=True)

In [106]:
movies.columns

Index(['imdbId', 'spanishTitle', 'original_title', 'year', 'certificate',
       'duration', 'directors', 'writers', 'stars', 'genres', 'countries',
       'companies', 'imdb_rating', 'metascore', 'popularity', 'awards',
       'budgetString', 'grossUsCanada', 'openingUsCanada', 'grossWorld',
       'budgetCurrency', 'budgetAmount', 'LOCATION', 'Value', 'budget'],
      dtype='object')

In [107]:
# Selección de columnas de imdb
imdb_merged = imdb_merged[['imdbId', 'englishTitle', 'originalTitle', 'isAdult', 'runtimeMinutes', 'genres', 'ratingImdb', 'numVotes']]

In [108]:
# Selección de columnas del escrapeado
movies = movies[['imdbId', 'spanishTitle', 'year', 'certificate',
                'directors', 'writers', 'stars', 'countries', 'companies',
                'metascore', 'awards', 'budget', 'grossUsCanada',
                'openingUsCanada', 'grossWorld']]

### Merge final para este año

In [109]:
movies_merged = pd.merge(imdb_merged, movies, on='imdbId', how='inner')
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15851 entries, 0 to 15850
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           15851 non-null  object 
 1   englishTitle     15851 non-null  object 
 2   originalTitle    15851 non-null  object 
 3   isAdult          15851 non-null  float64
 4   runtimeMinutes   13086 non-null  float64
 5   genres           15359 non-null  object 
 6   ratingImdb       8897 non-null   float64
 7   numVotes         8897 non-null   float64
 8   spanishTitle     15851 non-null  object 
 9   year             15851 non-null  float64
 10  certificate      3427 non-null   object 
 11  directors        15521 non-null  object 
 12  writers          14497 non-null  object 
 13  stars            14497 non-null  object 
 14  countries        15515 non-null  object 
 15  companies        11014 non-null  object 
 16  metascore        871 non-null    float64
 17  awards      

In [110]:
# Reordenar columnas

movies_merged = movies_merged[['imdbId', 'year', 'spanishTitle', 'originalTitle',
                               'englishTitle', 'ratingImdb', 'numVotes', 'metascore',
                               'isAdult', 'certificate', 'runtimeMinutes', 'genres', 'directors',
                               'writers', 'stars', 'countries', 'companies', 'awards',
                               'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld']]

In [111]:
movies_merged

,imdbId,year,spanishTitle,originalTitle,englishTitle,ratingImdb,numVotes,metascore,isAdult,certificate,...,directors,writers,stars,countries,companies,awards,budget,grossUsCanada,openingUsCanada,grossWorld
0,tt0016906,2014.0,Frivolinas,Frivolinas,Frivolinas,5.6,15.0,NaN,0.0,Not Rated,...,Arturo Carballo,"José López Alonso,Juan Belmonte,María Caballé","José López Alonso,Juan Belmonte,María Caballé",Spain,Ediciones Seleccionadas Arturo Carballo,NaN,NaN,NaN,NaN,NaN
1,tt0329539,2014.0,Rice Girl,Rice Girl,Rice Girl,5.8,86.0,NaN,0.0,NaN,...,Michael Fischa,Cat Ling,Cat Ling,United States,"Alpha-O Production,Rice Girl Productions LLC",Awards\n1 nomination,NaN,NaN,NaN,NaN
2,tt0330987,2014.0,"Tiden är en dröm, del 2","Tiden är en dröm, del 2","Tiden är en dröm, del 2",7.2,13.0,NaN,0.0,NaN,...,Jan Lindqvist,"Maths Isacson,Jan Lindqvist","Maths Isacson,Jan Lindqvist",Sweden,"Studio Ladan AB,Svenska Filminstitutet (SFI)",NaN,NaN,NaN,NaN,NaN
3,tt0338258,2014.0,Instruments of Darkness,Instruments of Darkness,Instruments of Darkness,6.6,25.0,NaN,0.0,NaN,...,Vincent Regan,"Vincent Regan,William Shakespeare","Vincent Regan,William Shakespeare","United Kingdom,France",Hell Kite Films,NaN,8.885525e+06,NaN,NaN,NaN
4,tt0365907,2014.0,Caminando entre las tumbas,A Walk Among the Tombstones,A Walk Among the Tombstones,6.5,114950.0,57.0,0.0,18,...,Scott Frank,"Lawrence Block,Scott Frank","Lawrence Block,Scott Frank",United States,"1984 Private Defense Contractors,Cross Creek P...",Awards\n1 nomination,2.800000e+07,26307600.0,12758780.0,58834384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15846,tt9878814,2014.0,La neve nera di Oslo,La neve nera di Oslo,La neve nera di Oslo,NaN,NaN,NaN,0.0,NaN,...,Paolo Marzoni,"Angelo Ferracuti,Paolo Marzoni","Angelo Ferracuti,Paolo Marzoni",Italy,Maxman Coop Società Cooperativa,NaN,NaN,NaN,NaN,NaN
15847,tt9890070,2014.0,Maaya,Maaya,Maaya,6.9,34.0,NaN,0.0,NaN,...,G. Neelakanta Reddy,"Harshvardhan Rane,Avantika Mishra,Nagendra Babu","Harshvardhan Rane,Avantika Mishra,Nagendra Babu",India,Shirdi Sai Combines,NaN,1.106022e+06,NaN,NaN,NaN
15848,tt9890106,2014.0,Boy Meets Girl (Tholiprema Katha),Boy Meets Girl (Tholiprema Katha),Boy Meets Girl (Tholiprema Katha),NaN,NaN,NaN,0.0,NaN,...,Dayakar Vasanth,NaN,NaN,India,Team 24 Cinemass,NaN,NaN,NaN,NaN,NaN
15849,tt9901656,2014.0,Entire Life and One Day,Entire Life and One Day,Entire Life and One Day,NaN,NaN,NaN,0.0,NaN,...,Davit Gujabidze,Davit Gujabidze,Davit Gujabidze,Georgia,Digital Kitchen Studio,NaN,NaN,NaN,NaN,NaN


## Feature engineering. Columnas `profit` y `roi`
Los estadísticos están distorsionados por los valores nulos que se eliminarán después

In [112]:
movies_merged['profit'] = movies_merged.grossWorld - movies_merged.budget
movies_merged['profit'].describe()

count    6.390000e+02
mean     2.985136e+07
std      1.008378e+08
min     -8.284920e+07
25%     -2.585541e+06
50%     -3.888280e+05
75%      6.379806e+06
max      8.940541e+08
Name: profit, dtype: float64

In [113]:
movies_merged['roi'] = (movies_merged.grossWorld - movies_merged.budget) / movies_merged.budget
movies_merged['roi'].describe()

count       639.000000
mean       1671.234340
std       32112.528855
min          -0.999950
25%          -0.903712
50%          -0.429128
75%           1.453080
max      746324.295632
Name: roi, dtype: float64

---
# Inspección inicial de correlaciones
## Registros no nulos con los que hacer las correlaciones de valoraciones y recaudación
Valoraciones: imdb_rating, metascore

Recaudación: budget, gross_world


In [114]:
print('Registros con ratingImdb:', len(movies_merged[movies_merged.ratingImdb.notnull()]))
print('Registros con metascore:', len(movies_merged[movies_merged.metascore.notnull()]))

print('Registros con budget:', len(movies_merged[movies_merged.budget.notnull()]))
print('Registros con grossUsCanada:', len(movies_merged[movies_merged.grossUsCanada.notnull()]))
print('Registros con openingUsCanada:', len(movies_merged[movies_merged.openingUsCanada.notnull()]))
print('Registros con grossWorld:', len(movies_merged[movies_merged.grossWorld.notnull()]))

print('\nRegistros con las rating, metascore, budget, grossWorld:',len(movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]))

Registros con ratingImdb: 8897
Registros con metascore: 871
Registros con budget: 4352
Registros con grossUsCanada: 653
Registros con openingUsCanada: 592
Registros con grossWorld: 2086

Registros con las rating, metascore, budget, grossWorld: 306


In [115]:
movies_merged = movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 4 to 12234
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           306 non-null    object 
 1   year             306 non-null    float64
 2   spanishTitle     306 non-null    object 
 3   originalTitle    306 non-null    object 
 4   englishTitle     306 non-null    object 
 5   ratingImdb       306 non-null    float64
 6   numVotes         306 non-null    float64
 7   metascore        306 non-null    float64
 8   isAdult          306 non-null    float64
 9   certificate      304 non-null    object 
 10  runtimeMinutes   306 non-null    float64
 11  genres           306 non-null    object 
 12  directors        306 non-null    object 
 13  writers          306 non-null    object 
 14  stars            306 non-null    object 
 15  countries        306 non-null    object 
 16  companies        304 non-null    object 
 17  awards        

## Correlaciones de valoraciones y recaudación

In [116]:
movies_merged.describe().round(2)

,year,ratingImdb,numVotes,metascore,isAdult,runtimeMinutes,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
count,306.0,306.00,306.00,306.00,306.0,306.00,3.060000e+02,2.690000e+02,2.620000e+02,3.060000e+02,3.060000e+02,306.00
mean,2014.0,6.28,104319.29,52.09,0.0,105.82,2.837284e+07,3.958940e+07,1.230233e+07,8.937739e+07,6.100455e+07,2.33
std,0.0,1.04,185821.89,19.16,0.0,16.32,4.282614e+07,6.446178e+07,2.076937e+07,1.747664e+08,1.379080e+08,8.39
min,2014.0,1.40,49.00,1.00,0.0,78.00,1.992750e+03,6.070000e+02,1.620000e+02,3.496000e+03,-3.221738e+07,-1.00
25%,2014.0,5.70,7933.50,38.00,0.0,94.00,4.500000e+06,1.699760e+05,3.749875e+04,8.184620e+05,-3.556467e+06,-0.81
50%,2014.0,6.40,31355.00,53.00,0.0,103.00,1.200000e+07,1.209630e+07,1.478536e+06,1.453161e+07,1.690503e+06,0.20
75%,2014.0,6.90,113317.00,67.00,0.0,114.00,2.950000e+07,5.057741e+07,1.587808e+07,8.614466e+07,4.798927e+07,2.32
max,2014.0,8.60,1577934.00,100.00,0.0,169.00,2.500000e+08,3.501264e+08,1.218976e+08,1.104054e+09,8.940541e+08,88.18


In [117]:
method = 'pearson'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2014 Corr Método: pearson


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.746867,0.196288,0.253828,0.260713,0.008084
metascore,0.746867,1.000000,0.069261,0.149602,0.168077,0.050980
budget,0.196288,0.069261,1.000000,0.892420,0.820393,-0.013869
grossWorld,0.253828,0.149602,0.892420,1.000000,0.990135,0.133109
profit,0.260713,0.168077,0.820393,0.990135,1.000000,0.172992
roi,0.008084,0.050980,-0.013869,0.133109,0.172992,1.000000


In [118]:
method = 'kendall'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2014 Corr Método: kendall


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.557089,0.079012,0.146355,0.167456,0.173721
metascore,0.557089,1.000000,-0.032600,0.069239,0.125381,0.131897
budget,0.079012,-0.032600,1.000000,0.618888,0.247797,0.265583
grossWorld,0.146355,0.069239,0.618888,1.000000,0.631587,0.649245
profit,0.167456,0.125381,0.247797,0.631587,1.000000,0.619501
roi,0.173721,0.131897,0.265583,0.649245,0.619501,1.000000


In [119]:
method = 'spearman'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2014 Corr Método: spearman


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.738598,0.113152,0.212802,0.247938,0.250962
metascore,0.738598,1.000000,-0.055337,0.098382,0.182230,0.194591
budget,0.113152,-0.055337,1.000000,0.798208,0.425160,0.374586
grossWorld,0.212802,0.098382,0.798208,1.000000,0.803127,0.830192
profit,0.247938,0.182230,0.425160,0.803127,1.000000,0.835558
roi,0.250962,0.194591,0.374586,0.830192,0.835558,1.000000


In [120]:
movies_merged[['ratingImdb', 'metascore', 'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld', 'profit', 'roi']].corr()

,ratingImdb,metascore,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
ratingImdb,1.000000,0.746867,0.196288,0.237735,0.136485,0.253828,0.260713,0.008084
metascore,0.746867,1.000000,0.069261,0.134330,0.040763,0.149602,0.168077,0.050980
budget,0.196288,0.069261,1.000000,0.782829,0.798362,0.892420,0.820393,-0.013869
grossUsCanada,0.237735,0.134330,0.782829,1.000000,0.903485,0.932390,0.938043,0.143071
openingUsCanada,0.136485,0.040763,0.798362,0.903485,1.000000,0.900293,0.892306,0.125773
grossWorld,0.253828,0.149602,0.892420,0.932390,0.900293,1.000000,0.990135,0.133109
profit,0.260713,0.168077,0.820393,0.938043,0.892306,0.990135,1.000000,0.172992
roi,0.008084,0.050980,-0.013869,0.143071,0.125773,0.133109,0.172992,1.000000


---
# Guardado dataset a pickle

In [121]:
import pickle

with open('../data/web_imdb_clean/movies_2014.pickle', 'wb') as f:
    pickle.dump(movies_merged, f)